In [28]:
from PIL import Image
from osgeo import gdal
from tifffile import imread, imsave

def to_transparent_png(foreground,background):
    
    # slice missing pixels of NN output    
    full_background = gdal.Open(background)
    background_array = full_background.ReadAsArray()
    cropped_background = background_array[:, 2:-2, 2:-2]
    
    # save cropped background as .png
    imsave('{}_cropped.png'.format(background[:len(background)-4]), cropped_background)

    # get data of the foreground image
    img1 = Image.open(foreground)
    img1 = img1.convert('RGBA')
    data_img1 = img1.getdata()

    # everything black in the foreground image becomes transparent    
    colors = []

    newData = []
    for item in data_img1:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)

    # convert foreground jpg to png    
    img1.putdata(newData)
    img1.save('{}.png'.format(foreground[:-4]))

    # open foreground and background png's  
    background = Image.open('{}_cropped.png'.format(background[:-4]))
    foreground = Image.open('{}.png'.format(foreground[:-4]))

    # paste images on each other    
    background.paste(foreground, (0, 0), foreground)
    background.save('output images/stacked_satellite_images.png')

to_transparent_png('output images/jpgtestsmall.jpg',"output images/x=0-2500_y=2500-5000.jpg")